In [204]:
import customtkinter as ctk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

from konlpy.tag import Okt
from tkinter import filedialog

import os
from glob import glob
import pandas as pd
import csv

In [165]:
# 리뷰 파일 로드
def load_excel(input_path:str):
    file_list = glob(os.path.join(input_path,"*.xlsx"))
    return pd.read_excel(file_list[0])

# 형태소 분석(명사 추출)
def tokenize(text:str):
    from konlpy.tag import Okt
    okt = Okt()
    pos = okt.pos(text, stem=True)
    tokens = [word for word, tag in pos if tag in ['Noun', 'Adjective']]
    return tokens

# 불용어 제거
def clean_text(nouns_text:list) -> list():
    # 불용어 파일 로드
    with open('stop_words_kr.txt', encoding='utf-8') as f:
        stop_words = f.read()
        stop_words = set(w.strip() for w in stop_words.split('\n') if w.strip())

    # 불용어를 제외하고 반환
    return [ noun for noun in nouns_text if noun not in stop_words]

def preprocess(input_file:pd.DataFrame) -> np.ndarray:
    nouns_text = input_file['REVIEW'].apply(lambda text: tokenize(text))
    removed_stopwords = nouns_text.apply(lambda nouns_text: clean_text(nouns_text))
    docs = removed_stopwords.apply(lambda word: ' '.join(word))
    return docs

def extract_top_keywords(corpus, top_n=10):
    """
    corpus : list[str] 또는 pandas.Series[str]
        문서 전체 (문서별 하나의 문자열)
    top_n : int
        상위 키워드 개수
    """
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform(corpus)
    feature_names = vectorizer.get_feature_names_out()
    
    # 전체 문서 기준 TF-IDF 합산
    scores = tfidf.toarray().sum(axis=0)
    top_indices = scores.argsort()[::-1][:top_n]

    top_keywords = [(feature_names[i], scores[i]) for i in top_indices]
    top_keywords = np.array(top_keywords)
    return top_keywords[:, 0], top_keywords[:,1]

# 저장
def save_to_csv(keywords, scores:None, filename="result.csv"):
    with open(filename, "w", newline="", encoding='utf-8-sig') as f:
        writer = csv.writer(f, delimiter=',')
        writer.writerow(["키워드", "중요도"])
        if scores:
            for word, score in zip(keywords, scores):
                writer.writerow([word, f"{score:.4f}"])
        else:
            for word in keywords:
                writer.writerow([word])
                
def main():
    input_path="input_text" # 후기 데이터 입력 폴더경로
    input_file = load_excel(input_path)
    input_file.head()
    
    docs = preprocess(input_file)
    keywords, scores = extract_top_keywords(docs)
    scores = list(map(float, scores))
    keywords = list(map(str, keywords))

In [201]:
save_to_csv(keywords, scores)

In [112]:
# # view scores for each documents
# import numpy as np

# feature_names = vectorizer.get_feature_names_out()
# tfidf_array = tfidf.toarray()

# for i, doc_vec in enumerate(tfidf_array):
#     print(f"문서 {i+1}")
#     for word, score in zip(feature_names, doc_vec):
#         if score > 0:
#             print(f"  {word}: {score:.3f}")


문서 1
  답변: 0.349
  상담사: 0.276
  오늘: 0.483
  응대: 0.361
  직원: 0.521
  콜센터: 0.409
문서 2
  기분: 0.854
  상담사: 0.520
문서 3
  고객감동: 0.486
  박경미: 0.486
  상담사: 0.234
  서비스: 0.486
  실현: 0.486
문서 4
  답변: 0.345
  상담: 0.358
  신분: 0.569
  친절: 0.478
  칭찬: 0.449
문서 5
  공휴일: 0.481
  근무: 0.481
  대한민국: 0.404
  칭찬: 0.379
  화이팅: 0.481
문서 6
  공공기관: 0.380
  모든: 0.410
  사이다: 0.453
  정말: 0.338
  정보: 0.514
  콜센터: 0.322
문서 7
  성함: 0.766
  안내: 0.643
문서 8
  관리: 0.219
  답변: 0.337
  방법: 0.278
  병원: 0.278
  보건소: 0.278
  상세: 0.504
  서로: 0.278
  응대: 0.175
  지역: 0.278
  질문: 0.233
  질병: 0.189
  질병예방: 0.278
문서 9
  고열: 0.363
  드림: 0.363
  불구: 0.329
  상태: 0.363
  새벽: 0.363
  전화: 0.329
  정보: 0.206
  정신: 0.363
  제공: 0.286
문서 10
  공공기관: 1.000
문서 11
  덕분: 0.373
  도움: 0.350
  상담사: 0.427
  이유: 0.444
  작성: 0.444
  통화: 0.402
문서 12
  내용: 0.445
  만족: 0.445
  매우: 0.367
  절차: 0.491
  정보: 0.279
  제공: 0.387
문서 13
  민원: 0.537
  설명: 0.498
  이해: 0.592
  정보: 0.336
문서 14
  경우: 0.197
  관리: 0.156
  기억: 0.166
  답변: 0.120
  부분: 0.179
  사람: 0.179
  상

In [104]:
# vectorizer = TfidfVectorizer()
# tfidf = vectorizer.fit_transform(docs)
# vectorizer.get_feature_names_out()

array(['가지', '감명', '감사', '개선', '거듭', '걱정', '경우', '경황', '계속', '고객', '고객감동',
       '고열', '공공기관', '공휴일', '관리', '관해', '국가', '국민', '궁금증', '규리', '근래',
       '근무', '기간', '기관', '기분', '기억', '김규리', '김예지', '김은빈', '내용', '노심초사',
       '놀람', '다시', '답변', '대기', '대한', '대한민국', '대해', '덕분', '뎅기열', '도움',
       '두서', '드림', '라면', '롤모델', '마음', '만족', '말씀', '매우', '모든', '문의',
       '문의사항', '문제', '민원', '박경미', '발음', '방금', '방법', '병역', '병원', '보건소',
       '보이지', '부분', '부장', '불구', '불만', '불이익', '사람', '사이다', '사항', '상담',
       '상담사', '상사', '상세', '상태', '새벽', '생각', '서로', '서비스', '설명', '성은', '성함',
       '셧던', '수고', '스타일', '승진', '시스템', '신분', '신속', '실현', '아주', '안나', '안내',
       '안심', '약속', '에러', '여기저기', '여러가지', '여행', '연결', '연락', '예방접종', '오늘',
       '우리나라', '운영', '원래', '위해', '응대', '의료', '의식', '이름', '이분', '이유',
       '이집트', '이해', '이해도', '인력', '자부심', '작성', '장시간', '전달', '전문', '전문성',
       '전염', '전화', '절차', '정도', '정말', '정보', '정신', '제공', '제반', '주심', '증상',
       '지금', '지역', '지연', '직원', '진급', '진상', '진짜', '진행', '질문', '질병',